# Search PubMed and Save Results

This example demonstrates the typical workflow to query pubmed and store
the results. The following backends are supported for storing the results:
* MySQL
* SQLite
* Citation (endnote/bibtex)
* DataFrames

### Set Up

In [1]:
using BioMedQuery.DBUtils
using BioMedQuery.PubMed
using BioMedQuery.Processes
using DataFrames
using MySQL
using SQLite

Variables used to search PubMed

In [2]:
email = ""; # Only needed if you want to contact NCBI with inqueries
search_term = """(obesity[MeSH Major Topic]) AND ("2010"[Date - Publication] : "2012"[Date - Publication])""";
max_articles = 5;
results_dir = ".";
verbose = true;

### MySQL backend

Initialize database, if it exists it connects to it, otherwise it creates it

In [3]:
const mysql_conn = DBUtils.init_mysql_database("127.0.0.1", "root", "", "pubmed_obesity_2010_2012");

Creates (and deletes if they already exist) all tables needed to save a pubmed search

In [4]:
PubMed.create_tables!(mysql_conn);

Search pubmed and save results to database

In [5]:
Processes.pubmed_search_and_save!(email, search_term, max_articles, mysql_conn, verbose)

Getting 5 articles, starting at index 0
------ESearch--------
------EFetch--------
------Save to database--------
Saving 5 articles to database
Finished searching, total number of articles: 5


#### Access all PMIDs

In [6]:
all_pmids(mysql_conn)

5-element Array{Int32,1}:
 24315250
 24444198
 24533500
 24694474
 25548090

#### Explore tables
You may use the MySQL command directly. If you want the return type to be a DataFrame, you need to explicitly request so.

In [7]:
tables = ["author_ref", "mesh_desc", "mesh_qual", "mesh_heading"]
for t in tables
    query_str = "SELECT * FROM $t LIMIT 5;"
    q = MySQL.query(mysql_conn, query_str, DataFrame)
    println(q)
end

5×9 DataFrames.DataFrame. Omitted printing of 3 columns
│ Row │ pmid     │ last_name │ first_name │ initials │ suffix  │ orcid   │
├─────┼──────────┼───────────┼────────────┼──────────┼─────────┼─────────┤
│ 1   │ 25548090 │ So        │ Eun Sun    │ ES       │ missing │ missing │
│ 2   │ 25548090 │ Yoo       │ Kwang Soo  │ KS       │ missing │ missing │
│ 3   │ 24694474 │ Sakurai   │ Masaru     │ M        │ missing │ missing │
│ 4   │ 24694474 │ Nakamura  │ Koshi      │ K        │ missing │ missing │
│ 5   │ 24694474 │ Miura     │ Katsuyuki  │ K        │ missing │ missing │
5×3 DataFrames.DataFrame
│ Row │ uid  │ name                 │ ins_dt_time         │
├─────┼──────┼──────────────────────┼─────────────────────┤
│ 1   │ 328  │ Adult                │ 2018-09-24T14:02:22 │
│ 2   │ 368  │ Aged                 │ 2018-09-24T14:02:22 │
│ 3   │ 369  │ Aged, 80 and over    │ 2018-09-24T14:02:22 │
│ 4   │ 704  │ Analysis of Variance │ 2018-09-24T14:02:22 │
│ 5   │ 1835 │ Body Weight        

In [8]:
MySQL.disconnect(mysql_conn);

### SQLite backend

In [9]:
const db_path = "$(results_dir)/pubmed_obesity_2010_2012.db";

Overwrite the database if it already exists

In [10]:
if isfile(db_path)
    rm(db_path)
end

Connect to the database

In [11]:
const conn_sqlite = SQLite.DB(db_path);

Creates (and deletes if they already exist) all tables needed to save a pubmed search

In [12]:
PubMed.create_tables!(conn_sqlite);

Search PubMed and save the results

In [13]:
Processes.pubmed_search_and_save!(email, search_term, max_articles, conn_sqlite, verbose)

Getting 5 articles, starting at index 0
------ESearch--------
------EFetch--------
------Save to database--------
Saving 5 articles to database
Finished searching, total number of articles: 5


#### Access all PMIDs

In [14]:
all_pmids(conn_sqlite)

5-element Array{Union{Missing, Int64},1}:
 24315250
 24444198
 24533500
 24694474
 25548090

#### Explore the tables
You may use the SQLite commands directly. The return type is a DataFrame.

In [15]:
tables = ["author_ref", "mesh_desc", "mesh_qual", "mesh_heading"]
for t in tables
    query_str = "SELECT * FROM $t LIMIT 5;"
    q = SQLite.query(conn_sqlite, query_str)
    println(q)
end

5×9 DataFrames.DataFrame. Omitted printing of 3 columns
│ Row │ pmid     │ last_name │ first_name │ initials │ suffix  │ orcid   │
├─────┼──────────┼───────────┼────────────┼──────────┼─────────┼─────────┤
│ 1   │ 25548090 │ So        │ Eun Sun    │ ES       │ missing │ missing │
│ 2   │ 25548090 │ Yoo       │ Kwang Soo  │ KS       │ missing │ missing │
│ 3   │ 24694474 │ Sakurai   │ Masaru     │ M        │ missing │ missing │
│ 4   │ 24694474 │ Nakamura  │ Koshi      │ K        │ missing │ missing │
│ 5   │ 24694474 │ Miura     │ Katsuyuki  │ K        │ missing │ missing │
5×3 DataFrames.DataFrame
│ Row │ uid   │ name              │ ins_dt_time         │
├─────┼───────┼───────────────────┼─────────────────────┤
│ 1   │ 12016 │ Reference Values  │ 2018-09-24 14:02:33 │
│ 2   │ 56910 │ Republic of Korea │ 2018-09-24 14:02:33 │
│ 3   │ 12372 │ ROC Curve         │ 2018-09-24 14:02:33 │
│ 4   │ 5221  │ Fatigue           │ 2018-09-24 14:02:33 │
│ 5   │ 9765  │ Obesity           │ 2018-09-24

### Citations
Citation type can be "endnote" or "bibtex"

In [16]:
enw_file = "$(results_dir)/pubmed_obesity_2010_2012.enw"
endnote_citation = PubMed.CitationOutput("endnote", enw_file, true)
Processes.pubmed_search_and_save!(email, search_term, max_articles, endnote_citation, verbose);

println(readstring(enw_file))

Getting 5 articles, starting at index 0
------ESearch--------
------EFetch--------
------Save to database--------
Saving citation for 5 articles
Finished searching, total number of articles: 5
┌ Warning: `readstring(filename::AbstractString)` is deprecated, use `read(filename, String)` instead.
│   caller = top-level scope at none:0
└ @ Core none:0
%0 Journal Article
%A So, ES
%A Yoo, KS
%D 2015
%T Waist circumference cutoff points for central obesity in the Korean elderly population.
%J J Appl Gerontol
%V 34
%N 1
%P 102-17
%M 25548090
%U http://www.ncbi.nlm.nih.gov/pubmed/25548090
%X ["The aim is to determine the appropriate cutoff values of waist circumference (WC) for an increased risk of the metabolic syndrome in the Korean elderly population. We analyzed the WC cutoff values of four groups divided according to sex and age with a total of 2,224 elderly participants aged 65 years old and above from the Fourth Korean National Health and Nutrition Examination Survey using the receiver

### DataFrames
Returns a dictionary of dataframes which match the content and structure of the database tables.

In [17]:
dfs = Processes.pubmed_search_and_parse(email, search_term, max_articles, verbose)

Getting 5 articles, starting at index 0
------ESearch--------
------EFetch--------
------Save to dataframes--------


Dict{String,DataFrames.DataFrame} with 8 entries:
  "basic"               => 5×13 DataFrames.DataFrame. Omitted printing of 9 col…
  "mesh_desc"           => 52×2 DataFrames.DataFrame…
  "mesh_qual"           => 9×2 DataFrames.DataFrame…
  "pub_type"            => 10×3 DataFrames.DataFrame…
  "abstract_full"       => 5×2 DataFrames.DataFrame. Omitted printing of 1 colu…
  "author_ref"          => 35×8 DataFrames.DataFrame. Omitted printing of 3 col…
  "mesh_heading"        => 78×5 DataFrames.DataFrame…
  "abstract_structured" => 4×4 DataFrames.DataFrame. Omitted printing of 1 colu…

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*